# Import libraries

In [ ]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load dataset

In [ ]:
df = pd.read_csv('/kaggle/input/datadir-celeb-a/aligned_celeba.txt', sep='\t')
df['datadir'] = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/' + df['datadir'].astype(str)
train_df, test_df = train_test_split(df, test_size=0.1)

# Model

In [ ]:
defaults = {
    'epochs': 20,
    'batch_size': 128,
    'fc1_num_neurons': 512,
    'fc2_num_neurons': 512,
    'fc3_num_neurons': 512,
    'seed': 7,
    'learning_rate': 3e-4,
    'optimizer': 'adam',
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

wandb.init(config=defaults, resume=True, name='For Tuned Model', project='CelebA Runs', notes='pretraining for tuned model, 0.1 test split')
config = wandb.config

# Load images into keras image generator 
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
datagen_test = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

# Define model
mobile_net_v2 = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(224,224,3),
)
mobile_net_v2.trainable = True

fc1 = tf.keras.layers.Dense(config.fc1_num_neurons,activation=config.hidden_activation)
fc2 = tf.keras.layers.Dense(config.fc2_num_neurons,activation=config.hidden_activation)
fc3 = tf.keras.layers.Dense(config.fc3_num_neurons,activation=config.hidden_activation)
bn1 = tf.keras.layers.BatchNormalization()
bn2 = tf.keras.layers.BatchNormalization()
bn3 = tf.keras.layers.BatchNormalization()
bn4 = tf.keras.layers.BatchNormalization()

model = tf.keras.models.Sequential([
    mobile_net_v2,
    tf.keras.layers.Flatten(),
    bn1,
    fc1,
    bn2,
    fc2,
    bn3,
    fc3,
    bn4,
    tf.keras.layers.Dense(1, activation=config.output_activation),
])
model.summary()

# Compile model 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=config.loss_function,
    metrics=config.metrics,
)

model.fit(
    train_generator,
    validation_data=test_generator,
    shuffle=True,
    epochs=config.epochs,
    callbacks=[WandbCallback()],
)
model.save_weights('model_celeba_tune.h5') 
# run.finish()